# Course 4: Planning
## Part 1: Behaviour Planning in C++
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives
* TODO.

## 1. Introduction

TODO.

## 2. Programming Task

### 2.1. Implement a Cost Function in C++

In previous quizzes, you designed a cost function to choose a lane when trying to reach a goal in the highway driving scenario:
$$
\begin{align}
\mathrm{cost} = 1 - \mathrm{e}^{-\frac{\vert \Delta d \vert}{\Delta s}}.
\end{align}
$$

Here, $\Delta d$ was the lateral distance between the goal lane and the final chosen lane, and $\Delta s$ was the longitudinal distance from the vehicle current position to the goal position. 

In this exercise, we will implement a cost function in C++ that also includes states for planning a lane change right or left (denoted $\mathrm{PLCR}$ and $\mathrm{PLCL}$, respectively). The cost function should incoporate these states. As input to the cost function, we have the following:
* **Intended lane**: the intended lane for the given behaviour. For the $\mathrm{PLCR}$, $\mathrm{PCLC}$, $\mathrm{LCR}$, and $\mathrm{LCL}$, this corresponds to the lane immediately adjacent to the current (one lane over);
* **Final lane**: the immediate resulting lane of the given behaviour. For $\mathrm{LCR}$ and $\mathrm{LCL}$, this would correspond to one lane over;
* **Distance to the goal**: $\Delta s$, the longitudinal distance from the vehicle to the goal;
* **Goal lane**: lane in which the goal marker is situated.

The goal of this exercise is to modify the $\vert \Delta d \vert$ in the equation above so that it satisfies:
* $\vert \Delta d \vert$ decreases when both the intended and final lane are closer to the goal lane;
* Each possible behaviour state $\mathrm{KL}$, $\mathrm{PLCR}$ / $\mathrm{PLCL}$, $\mathrm{LCR}$ / $\mathrm{LCL}$ has a unique cost;
* The values produced by the cost function are in range $[0, 1]$.  

In [1]:
// From J. Moran's `1_distance_cost.cc`

In [2]:
#include <cmath>


/* Computes the distance cost to the goal state including lane changes.
 *
 * Here the cost increases with both the distance of intended lane from the
 * goal and the distance of the final lane form the goal. The cost also 
 * becomes larger as the vehicle gets closer to approaching the goal (i.e.,
 * the cost increases as longitudinal distance to the goal decreases).
 *
 * @param    goal_lane          Lane number containing the goal marker. 
 * @param    intended_lane      Lane number inteded for given manouevre.
 * @param    final_lane         Lane number immediately following manouevre. 
 * @param    distance_to_goal   Longitudinal distance from vehicle to goal.
 * @returns  cost               Associated distance cost of planned manoeuvre.
 */
double goal_distance_cost(
    int goal_lane, 
    int intended_lane, 
    int final_lane, 
    double distance_to_goal
) {
  // Compute the weighted lateral remaining distance to the goal lane
  double delta_d = 2.0 * goal_lane - intended_lane - final_lane;
  // Taking absolute value to get magnitude of total remaining distance
  double cost = 1.0 - exp(-(std::abs(delta_d) / distance_to_goal));
  return cost;
}

#### Testing the distance cost function

In [3]:
// From J. Moran's `1_tests.cc`

In [5]:
#include <iostream>

/* Tests the goal-distance cost function.
 * The costs are given for input (`intended_lane`, `final_lane`, `goal_lane`).
 * 
 * For example, we might have the following lane configuration:
 *
 * --------------------------------------------------------------
 *  3 |
 * --------|-----------------------------------------------------
 *  2 |    o
 * --------|------------------|----------------------------------
 *  1 |    o                  o
 * -----------------------------------------------------|--------
 *  0 |                                                 x
 * --------------------------------------------------------------
 * 
 * Where we have a `goal_lane = 0`, and three possible manouevres to
 * score in `intended_lane = 2` and `intended_lane = 1`. The distance
 * from `intended_lane = 2` to the `goal_lane = 0` can be e.g., 10m, 
 * while the distances from `intended_lane = 1` to the `goal_lane = 0`
 * can be 10m and e.g., 5m. 
 * 
 * Using our cost function, we want to give the two manouevres in lane 1 
 * a lower cost score since they are closer laterally to the goal lane.
 * However, within lane 1, we want to give the manouevre farther away from
 * the goal marker a lesser cost score (i.e., cost decreases as longitudinal
 * distance to goal increases). This is to factor in reaction time.
 * 
 * Therefore, the manouevre the highest cost will be for (2, 2, 10), i.e.,
 * position in lane 2 with no intended or final lane change closer to goal.
 * Assuming an intention to move into lane 1, i.e., (2, 1, 10), this will
 * be our second highest cost. The next highest will be for (1, 1, 5),
 * which represents position in lane 1 with shorter distance to goal and no
 * intention of moving into goal lane. After that, we have (1, 0, 5), which
 * is the position in lane 1 with intention of moving into goal lane but with
 * shorter distance. This results in a lowest cost score assigned to
 * (1, 0, 10), which represents the position in lane 1 with larger distance to
 * goal and intention of moving into goal lane. This is the most optimal case.
 */
void test_goal_distance_cost() {
  // Integer number of the goal lane (see docstring for example)
  int goal_lane = 0;
  double cost;
  std::cout << "Costs for (intended_lane, final_lane, goal_distance):" << "\n";
  std::cout << "---------------------------------------------------------" << "\n";
  cost = goal_distance_cost(goal_lane, 2, 2, 1.0);
  std::cout << "The cost is " << cost << " for " << "(2, 2, 1.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 2, 2, 10.0);
  std::cout << "The cost is " << cost << " for " << "(2, 2, 10.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 2, 2, 100.0);
  std::cout << "The cost is " << cost << " for " << "(2, 2, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 1, 2, 100.0);
  std::cout << "The cost is " << cost << " for " << "(1, 2, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 1, 1, 100.0);
  std::cout << "The cost is " << cost << " for " << "(1, 1, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 0, 1, 100.0);
  std::cout << "The cost is " << cost << " for " << "(0, 1, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 0, 0, 100.0);
  std::cout << "The cost is " << cost << " for " << "(0, 0, 100.0)" << "\n";
    
}

In [6]:
test_goal_distance_cost()

Costs for (intended_lane, final_lane, goal_distance):
---------------------------------------------------------
The cost is 0.981684 for (2, 2, 1.0)
The cost is 0.32968 for (2, 2, 10.0)
The cost is 0.0392106 for (2, 2, 100.0)
The cost is 0.0295545 for (1, 2, 100.0)
The cost is 0.0198013 for (1, 1, 100.0)
The cost is 0.00995017 for (0, 1, 100.0)
The cost is 0 for (0, 0, 100.0)
